In [188]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from enum import Enum
import re



In [189]:
# Set whether to display selium browser
VISIBLE = True

options = Options()
# If not visible do not display browser to user
if not VISIBLE: options.add_argument('--headless')

url = "https://goblin.bet/#/"
driver = webdriver.Firefox(options=options)
driver.get(url)




In [190]:
contButt = WebDriverWait(driver, timeout=30).until(lambda d: d.find_element(By.CLASS_NAME, "WelcomeStart.Left"))
contButt.click()

In [191]:
pageSrc = driver.page_source
soup = BeautifulSoup(pageSrc)

RIGHT = True
LEFT = False

def getLog(soup: BeautifulSoup):
    log = []
    # Find log
    logSoup = soup.find("div", {"class": "scrollhost BLogScroll"})

    # If print log true print out the log
    for lg in logSoup.find_all("div", {"class": "LogText"}):
        log.append(lg.text)
    # return the betting log
    return log

def getMoney(soup: BeautifulSoup):
    return soup.find('span', {"class": "BetsScore"}).text

def getCreatureSoup(soup: BeautifulSoup, side=LEFT):
    if side == LEFT:
        return soup.find("div", {"class": "Block Statsheet Left TeamRed"})
    return soup.find("div", {"class": "Block Statsheet Right TeamBlue"})
        

In [192]:
# Strips all alphabetic characters
def stripChrs(word):
    return re.sub("[^0-9]", '', word)

# Strips all nonalphanumeric characters in string
def stripInts(word):
    return re.sub(r'\W+', '', word)

# Finds a tag with a specfic class name
def findSpanClass(soup, className, all=False):
    if not all:
        return soup.find("span", {"class": className})
    return soup.find_all("span", {"class": className})


# Class containing all betting creature information 
class Creature:

    def __init__(self, soup: BeautifulSoup, side=LEFT):
        self.side = side
        self.soup = getCreatureSoup(soup, side)
        self.name = self.InitName(self.soup)
        self.desc, self.cr = self.InitDescStats(self.soup)
        self.InitStats()
        
    def InitName(self, soup: BeautifulSoup):
        return findSpanClass(soup, "SSName").text
    
    # Intializes description stats and cr rank
    def InitDescStats(self, soup: BeautifulSoup):
        cretInfo = None
        cr = None
        for i, tag in enumerate(soup.find_all('span', {'class': "SSInfo"})):
            if i == 0:
                cretInfo = tag.text
            if i == 1:
                cr = tag.text.split(' ')[1]
        return cretInfo, cr

    def getAction(self, action: BeautifulSoup):
        action = [findSpanClass(action, "ActName").text, findSpanClass(action, "ActDesc").text]
        return action

    # Initializes all stats in the creature sheet
    def InitStats(self):
        # Initialize stat categories
        self.stats = None
        self.immunities = None
        self.resists = None
        self.actions = None
        self.conditions = None
        self.wins = None

        # Gets list of all tags in the stats soup
        statsList =  self.soup.find("div", {"class": "SSStats"})
        # Go through every header value in the tags
        for head in statsList.find_all("span", {"class": "SSHeader"}):
            # Record the text of the header
            headTxt = stripInts(head.text)
            if headTxt == "ATTRIBUTES":
                self.stats = self.getAttributes(statsList)
            if headTxt == "WINS":
                self.wins = head.find_next_sibling('span').text.split(',')
            # If content is immunities
            if headTxt == "IMMUNE":
                # Store immunities using next tag
                self.immunities = [word.replace(' ', '') for word in head.find_next_sibling("span").text.split(',')]
            if headTxt == "RESIST":
                # Store resistances using next tag
                self.resists = [word.replace(' ', '') for word in head.find_next_sibling("span").text.split(',')]
            if headTxt == "ACTIONS":
                self.actions = []
                # Append each action to list
                self.InitActions(head)
            if headTxt == "CONDITIONS":
                # Find all conditions
                self.conditions =  self.getConditions(statsList)
            
    # Initializes possible actions of the 
    def InitActions(self, actionHead):
        try:
            child = actionHead
            for action in child.find_next_siblings("div"):
                self.actions.append(self.getAction(action))
            child = child.next
        except AttributeError:
            None

    
    def getConditions(self, statsList: BeautifulSoup):
        conditions = []
        # Highlighted condition
        for tag in findSpanClass(statsList, "Stat Small CanPop Feat"):
            conditions.append(tag.text)
            # Break as the next tag is redundant
            break
        # Loop through all remaining conditions
        for tag in findSpanClass(statsList, "Stat Small", all=True):
            conditions.append(tag.text)

        return conditions


    # Copies attributes tag
    def getAttributes(self, statsList: BeautifulSoup):
        stats = {"STR": None, "DEX": None, "CON": None, "INT": None, "WIS": None, "CHA": None}
        for i, stat in enumerate(statsList.find_all("span", {"class": "Stat"})):
            if i == 0:
                stats["STR"] = stripChrs(stat.text)
            elif i == 1:
                stats["DEX"] = stripChrs(stat.text)
            elif i == 2:
                stats["CON"] = stripChrs(stat.text)
            elif i == 3:
                stats["INT"] = stripChrs(stat.text)
            elif i == 4:
                stats["WIS"] = stripChrs(stat.text)
            elif i == 5:
                stats["CHA"] = stripChrs(stat.text)
            elif i == 6:
                hpVals = [val for val in re.split("/", stat.text)]
                stats["HP"] = stripChrs(hpVals[0])
                stats["HPMax"] = stripChrs(hpVals[1])
            elif i == 7:
                stats["AC"] = stripChrs(stat.text)
            elif i == 8:
                stats["SPD"] = stripChrs(stat.text)
        return stats

soup = BeautifulSoup(driver.page_source)
cret = Creature(soup, RIGHT)


In [193]:
def creatureLog(cret: Creature):
    print(f"Name: {cret.name}")
    print(f"Description: {cret.desc}")
    print(f"CR: {cret.cr}")
    print("Side: {}".format("Right" if cret.side == RIGHT else "Left"))
    print(f"Wins: {cret.wins}")
    print(f"Stats: {cret.stats}")
    print(f"Actions: {cret.actions}")
    print(f"Resistances: {cret.resists}")
    print(f"Immunitises: {cret.immunities}")
    print(f"Conditions: {cret.conditions}")
creatureLog(cret)

Name: Water Elemental
Description: Large Elemental Neutral
CR: 5
Side: Right
Wins: ['Gorgon']
Stats: {'STR': '18', 'DEX': '14', 'CON': '18', 'INT': '5', 'WIS': '10', 'CHA': '8', 'HP': '59', 'HPMax': '59', 'AC': '14', 'SPD': '30'}
Actions: [['Slam: ', '+6, 2d8+4 Bludgeoning'], ['Squeeze: ', '2d8+4 Bludgeoning (If Grappled & Restrained)'], ['Multiattack: ', 'Squeeze, 2 Slam'], ['Whelm: ', '2d8+4 Bludgeoning (STR Save)']]
Resistances: ['Bludgeoning', 'Piercing', 'Slashing', 'Acid']
Immunitises: ['Poison', 'Exhaustion', 'Grappled', 'Paralyzed', 'Petrified', 'Poisoned', 'Prone', 'Restrained', 'Asleep']
Conditions: ['Regenerator', 'Freezable']


In [230]:
import time

prevLog = []
while True:# Get Soup

    soup = BeautifulSoup(driver.page_source)

    # Initialize creatures
    cretR = Creature(soup, side=RIGHT)
    cretL = Creature(soup)
    # Gets current log
    log = getLog(soup)
    currentEntry = log[0]
    # Check if logIsEqual
    if prevLog == log or log == None or prevLog == None:
        continue

    
    print(currentEntry)

    prevLog = log
    time.sleep(1)


+ Bone Devil Rolls Initiative: 16
+ Bone Devil Wins Initiative!
+ Bone Devil runs 25 feet forwards.
+ Bone Devil attacks Cloud Giant with a Claw.
+ The Bone Devil's Claw misses!
+ Bone Devil attacks Cloud Giant with a Claw.
+ The Bone Devil's Claw hits!
+ Cloud Giant takes 9 Slashing damage!
+ Bone Devil attacks Cloud Giant with a Sting.
+ The Bone Devil's Sting hits!
+ Cloud Giant takes 14 Piercing damage!
+ Cloud Giant takes 18 Poison damage!
+ Cloud Giant saves against Poisoned.
+ Cloud Giant attacks Bone Devil with a Morningstar.
+ The Cloud Giant's Morningstar hits!
+ Bone Devil takes 23 Psychic damage!
+ Cloud Giant attacks Bone Devil with a Morningstar.
+ The Cloud Giant's Morningstar misses!
+ Bone Devil attacks Cloud Giant with a Claw.
+ The Bone Devil's Claw hits!


KeyboardInterrupt: 